<b>Categorical Variable: </b>recebe número limitado de valores; há algumas opções para aquela vairável (como uma pesquisa sobre "quão frequentemento você toma café" e dá 3 opções de escolha); erro caso tente colocar essas variáveis na maioria dos modelos de ML sem pré-processamento.

## 3 Abordagens
1. <b>Dropar variáveis categóricas:</b> só vai funcionar bem caso a coluna não contenha informações úteis
2. <b>Codificação Ordinal:</b> atribuir cada valor único para um número inteiro diferente; não necessariamente haverá um ordenamento no valor escolhido
3. <b>One-Hot Encoding: </b>cria novas colunas indicando a presença ou ausência de cada possível valor no dado original; traduzir a variável em uma coluna para cada valor possível

<b>Nominal Variables:</b> variáveis categórias sem ranking intrínseco

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("melb_data.csv")

y = data["Price"]
X = data.drop(["Price"], axis=1)

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [2]:
# removendo colunas com valores faltando
cols_with_missing = [
    col for col in X_train_full.columns if X_train_full[col].isnull().any()
]
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

# cardinalidade = numero de valores únicos em uma coluna

# selecionar colunas categóricas com cardinalidade relativamente baixa
low_cardinality_cols = [
    cname
    for cname in X_train_full.columns
    if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"
]

# selecionando colunas numéricas
numerical_cols = [
    cname
    for cname in X_train_full.columns
    if X_train_full[cname].dtype in ["int64", "float64"]
]

# manter colunas selecionadas apenas
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()


In [3]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


In [4]:
# list de variáveis categóricas
s = (X_train.dtypes == "object")
object_cols = list(s[s].index)

print("Categorical variables:\n ", object_cols)

Categorical variables:
  ['Type', 'Method', 'Regionname']


In [5]:
# função para definir qualidade de cada abordagem (score)
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train=y_train, y_valid=y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### 1) Drop das variáveis categóricas

In [6]:
drop_X_train = X_train.select_dtypes(exclude=["object"])
drop_X_valid = X_valid.select_dtypes(exclude=["object"])

print("MAE-1: \n ", score_dataset(drop_X_train, drop_X_valid))

MAE-1: 
  175703.48185157913


### 2) Ordinal Encoding

In [7]:
from sklearn.preprocessing import OrdinalEncoder

# copia para evitar perder os dados
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# aplicar codificação ordinal para cada coluna com dados categoricos
ordinal_encoder = OrdinalEncoder()

label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

print("MAE-2:\n ", score_dataset(label_X_train, label_X_valid))

### 3) One-hot Encoding
handle_unknown="ignore" para evitar erros quando os dados de validação contém classes que não são representaradas nos dados de treino<br>
sparse=False garante que as colunas codificadas são retornadas como array do Numpy (exceto matrizes esparsas)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):


TypeError: Feature names are only supported if all input features have string names, but your input has ['int', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.